<a href="https://colab.research.google.com/github/tomasborrella/TheValley/blob/main/notebooks/mds%2B5/spark02/3_Ejercicio_resuelto_Spark_DataFrames_funciones_y_transformaciones_basicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio resuelto de Spark DataFrames: Funciones y transformaciones básicas

Notebook por [Tomás Borrella Martín](https://www.linkedin.com/in/tomasborrella/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/1fpzzxMSECiA7bhcloTDLfvny-ZoM8k98cV0ZzZIAx-8/edit?usp=sharing)

# Ejercicio propuesto

Sobre unos dataset con tweets (tweets.json) obtener:

1.   Campos id, created_at y text de los diez primeros tweets del dataset.
2.   Campos id, name (del usuario) y location (del usuario).
3.   Los 10 usuarios con más amigos (campo friends_count).
4.   Los 10 usuarios con más tweets.

El dataset está subido a Google Drive: [Enlace](https://drive.google.com/file/d/18np5pJbHy1e1Dr1h17wTgZTb6-gJ23RJ/view?usp=sharing)

Para descargarlos se puede usar wget de esta forma:

```
# Descargamos los datos al entorno de Colab
!mkdir -p /content/data
!wget -O /content/data/tweets.json 'https://drive.google.com/uc?id=18np5pJbHy1e1Dr1h17wTgZTb6-gJ23RJ'
```



# 1. Instalación Spark

In [1]:
# Install JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Install Spark
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
!tar xf spark-3.2.4-bin-hadoop2.7.tgz

In [3]:
# Install findspark
!pip install -q findspark

In [4]:
# Environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"

In [5]:
# Find spark
import findspark
findspark.init()

In [ ]:
# PySpark 
!pip install pyspark==3.2.4

# 2. Spark Session
Punto de entrada de la aplicación de Spark

In [7]:
# Imports
from pyspark.sql import SparkSession

In [8]:
# Create Spark Session
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("Spark Dataframes")
         .getOrCreate()
)

# Datos

In [ ]:
# Descargamos los datos al entorno de Colab
!mkdir -p /content/data
!wget -O /content/data/tweets.json 'https://drive.google.com/uc?id=18np5pJbHy1e1Dr1h17wTgZTb6-gJ23RJ' 

In [10]:
df = spark.read.json('/content/data/tweets.json')

## Explorar datos

Mostramos información de su esquema con printSchema

In [ ]:
df.printSchema()

Mostramos los 5 primeros registros

In [ ]:
df.show(5)

# 1. Campos id, created_at y text de los diez primeros tweets del dataset

In [ ]:
(df
 .select(df['id'], df['created_at'], df['text'])
 .limit(10)
 .show()
)

# 2. Campos id, name (del usuario) y location (del usuario)

In [ ]:
df.select('user').printSchema()

In [ ]:
(df
 .select(df['id'], df['user']['name'], df['user']['location'])
 .limit(10)
 .show()
)

In [ ]:
(df
 .select(df['id'], df['user.name'], df['user.location'])
 .limit(10)
 .show()
)

# 3. Los 10 usuarios con más amigos (campo friends_count)

In [17]:
from pyspark.sql import functions as F

In [ ]:
(df
 .groupBy(df['user']['name'].alias('user_name'))
 .agg(F.max(df['user']['friends_count']).alias('friends_count_max'))
 .orderBy('friends_count_max', ascending=False)
 .limit(10)
 .show()
)

# Los 10 usuarios con más tweets

In [ ]:
(df
 .groupBy(df['user']['name'].alias('user_name'))
 .agg(F.count(df['user']['name']).alias('tweets_count'))
 .orderBy('tweets_count', ascending=False)
 .limit(10)
 .show()
)

# Spark Stop

In [20]:
spark.stop()